# Generación de un nuevo CSV de Tripletas basado en relaciones nutricionales e ingredientes  
*(Usando la columna `name` como identificador)*

El siguiente código hace lo siguiente:
1. Carga el CSV `RAW_recipes.csv` y elimina las filas sin nombre.
2. Convierte las columnas `ingredients` y `nutrition` de cadenas a listas Python.
3. Separa la lista nutricional en 7 columnas:
   - calories  
   - total_fat_pdv  
   - sugar_pdv  
   - sodium_pdv  
   - protein_pdv  
   - saturated_fat_pdv  
   - carbs_pdv  
4. Define funciones para clasificar cada valor nutricional en bajo, normal o alto, según los umbrales establecidos.
5. Genera tripletas:
   - Una tripleta por cada ingrediente, usando la relación `has_ingredient`.  
   - Una tripleta para cada nutriente, asignando la relación correspondiente (por ejemplo, `low_calories`, `normal_calories` o `high_calories`).
6. Guarda todas las tripletas en un archivo `new_triplets.csv`.

In [1]:
import pandas as pd
import ast
import numpy as np

# 1. Cargar el CSV y eliminar filas sin nombre
df = pd.read_csv("foodcom_data/RAW_recipes.csv")
df = df.dropna(subset=['name'])  # Elimina filas donde 'name' sea NaN


In [2]:

# 2. Convertir las columnas 'ingredients' y 'nutrition' de cadenas a listas
df['ingredient_list'] = df['ingredients'].apply(ast.literal_eval)
df['nutrition_list'] = df['nutrition'].apply(ast.literal_eval)


In [3]:

# 3. Separar la columna 'nutrition_list' en 7 columnas
nutri_cols = [
    'calories',
    'total_fat_pdv',
    'sugar_pdv',
    'sodium_pdv',
    'protein_pdv',
    'saturated_fat_pdv',
    'carbs_pdv'
]
df[nutri_cols] = pd.DataFrame(df['nutrition_list'].tolist(), index=df.index)


In [4]:

# 4. Función para clasificar valores nutricionales
def classify_value(value, thresholds, low_label, normal_label, high_label):
    """
    thresholds: tuple (low_threshold, high_threshold)
    - Si value < low_threshold -> low_label
    - Si low_threshold <= value <= high_threshold -> normal_label
    - Si value > high_threshold -> high_label
    """
    low_th, high_th = thresholds
    if value < low_th:
        return low_label, value
    elif value <= high_th:
        return normal_label, value
    else:
        return high_label, value

# Umbrales según tus reglas:
nutri_rules = {
    'calories': ((174, 520), "low_calories", "normal_calories", "high_calories"),
    'total_fat_pdv': ((8, 41), "low_fat", "normal_fat", "high_fat"),
    'sugar_pdv': ((9, 68), "low_sugar", "normal_sugar", "high_sugar"),
    'sodium_pdv': ((5, 33), "low_sodium", "normal_sodium", "high_sodium"),
    'protein_pdv': ((7, 51), "low_protein", "normal_protein", "high_protein"),
    'saturated_fat_pdv': ((7, 52), "low_saturated_fat", "normal_saturated_fat", "high_saturated_fat"),
    'carbs_pdv': ((4, 16), "low_carbs", "normal_carbs", "high_carbs")
}


In [5]:

# 5. Crear la lista de tripletas: [subject, relation, object]
triplets = []

# 5.1. Generar tripletas "has_ingredient" usando 'name' como identificador
for _, row in df.iterrows():
    # Asegurarse de que 'name' es una cadena (ya se eliminó NaN, pero por precaución)
    if not isinstance(row['name'], str):
        continue
    recipe_name = row['name'].strip().lower().replace(" ", "_")
    
    # Para cada ingrediente en la lista de ingredientes
    for ing in row['ingredient_list']:
        if isinstance(ing, str):
            ingredient = ing.strip().lower().replace(" ", "_")
            triplets.append([recipe_name, "has_ingredient", ingredient])
        else:
            continue

# 5.2. Generar tripletas nutricionales (por cada nutriente, se asigna la relación correspondiente)
for _, row in df.iterrows():
    if not isinstance(row['name'], str):
        continue
    recipe_name = row['name'].strip().lower().replace(" ", "_")
    for col in nutri_cols:
        thresholds, low_label, normal_label, high_label = nutri_rules[col]
        relation_label, value = classify_value(row[col], thresholds, low_label, normal_label, high_label)
        triplets.append([recipe_name, relation_label, str(value)])


In [6]:

# 6. Guardar las tripletas en un nuevo CSV
triplets_df = pd.DataFrame(triplets, columns=["subject", "relation", "object"])
triplets_df.to_csv("new_triplets.csv", index=False)